In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
import vdmlab as vdm
from analyze_maze import find_zones
from analyze_fields import get_unique_fields, categorize_fields

In [ ]:
from load_data import get_pos, get_spikes, get_lfp
import info.r068d2 as info

In [ ]:
pos = get_pos(info.pos_mat, info.pxl_to_cm)
lfp = get_lfp(info.good_swr[0])
spikes = get_spikes(info.spike_mat)

In [ ]:
times = info.task_times['phase3']

In [ ]:
sliced_spikes = [spiketrain.time_slice(times.start, times.stop) for spiketrain in spikes]
sliced_pos = pos.time_slice(times.start, times.stop)

In [ ]:
plt.plot(sliced_pos.x, sliced_pos.y, 'g.', ms=2)
plt.show()

In [ ]:
binsize = 3
xedges = np.arange(sliced_pos.x.min(), sliced_pos.x.max() + binsize, binsize)
yedges = np.arange(sliced_pos.y.min(), sliced_pos.y.max() + binsize, binsize)
tuning_curves = vdm.tuning_curve_2d(sliced_pos, sliced_spikes, xedges, yedges, gaussian_sigma=0.1)

In [ ]:
zones = find_zones(info)

field_thresh = 1.0
fields_tunings = categorize_fields(tuning_curves, zones, xedges, yedges, field_thresh=field_thresh)

In [ ]:
xx, yy = np.meshgrid(xedges, yedges)
for key in fields_tunings['u']:
    pp = plt.pcolormesh(xx, yy, fields_tunings['u'][key], cmap='YlGn')
    plt.colorbar(pp)
    plt.axis('off')
    plt.show()

In [ ]:
zones

In [ ]:
u_line = LineString(info.u_trajectory)
shortcut_line = LineString(info.shortcut_trajectory)
novel_line = LineString(info.novel_trajectory)

In [ ]:
fields_tunings['u'].keys()

In [ ]:
u_line.length

In [ ]:
u_line.bounds

In [ ]:
list(u_line.coords)

In [ ]:
for neuron in fields_tunings['u']:
    row = np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[0][0]
    col = np.where(fields_tunings['u'][neuron] == fields_tunings['u'][neuron].max())[1][0]
    fields_tunings['u'][neuron][row][col]

    p = Point(row, col)
    this_p = u_line.interpolate(u_line.project(p, normalized=True))

    plt.plot(this_p.xy[0], this_p.xy[1], 'c.', ms=10)
    closest_pt = list(p.coords)[0]
    print(closest_pt)
    plt.plot(closest_pt[0], closest_pt[1], 'g.', ms=10)

plt.plot(u_line.xy[0], u_line.xy[1], 'k.')
plt.show()

In [ ]:
p = Point(row, col) 

In [ ]:
u_line.project(p)

In [ ]:
this_p = u_line.interpolate(u_line.project(p))

In [ ]:
print(this_p)

In [ ]:
plt.plot(u_line.xy[0], u_line.xy[1], 'k.')
plt.plot(this_p.xy[0], this_p.xy[1], 'c.', ms=10)
plt.show()

# saving many tuning curves same plot

In [ ]:
import matplotlib.gridspec as gridspec

rows = 8
cols = 14
fig = plt.figure(figsize=(10, 10))

grids = gridspec.GridSpec(cols, rows)
grids.update(wspace=0.02, hspace=0.05)

xx, yy = np.meshgrid(xedges, yedges)
for i in range(rows*cols):
    ax = plt.subplot(grids[i])
    pp = ax.pcolormesh(xx, yy, tuning_curves[i], cmap='YlGn')
    ax.axis('off')

savepath = 'E:/code/emi_shortcut/plots/tuning/' + info.session_id + 'many_tuning-curves.png'
plt.savefig(savepath, dpi=300, bbox_inches='tight')